# Yfinance (capped at 10)

In [25]:
from dotenv import load_dotenv
import os
# OR.PA, AZN.L, NESN.SW

load_dotenv()
MARKETAUX_TOKEN = os.getenv("MARKETAUX_API_TOKEN", "YOUR_MARKETAUX_TOKEN")
FINNHUB_TOKEN = os.getenv("FINNHUB_API_TOKEN", "YOUR_FINNHUB_TOKEN")
NEWSDATA_API_KEY = os.getenv("NEWSDATA_API_KEY", "YOUR_NEWSDATA_API_KEY")

In [30]:
import yfinance as yf

apple = yf.Ticker("AAPL")

# These are the only functions to fetch news via yfinance (rate limited at 10 articles)
print(len(apple.news))
print(len(apple.get_news(count=10, tab="news")))
print(len(apple.get_news(count=20, tab="news")))

"""
### API Documentation -> #https://ranaroussi.github.io/yfinance/reference/api/yfinance.Ticker.get_news.html

Function: yfinance.Ticker.get_news

Ticker.get_news(count=10, tab='news', proxy=<object object>) → list
    Allowed options for tab: “news”, “all”, “press releases

Function: yfinance.Ticker.news

property Ticker.news: list
"""

10
10
10


"\n### API Documentation -> #https://ranaroussi.github.io/yfinance/reference/api/yfinance.Ticker.get_news.html\n\nFunction: yfinance.Ticker.get_news\n\nTicker.get_news(count=10, tab='news', proxy=<object object>) → list\n    Allowed options for tab: “news”, “all”, “press releases\n\nFunction: yfinance.Ticker.news\n\nproperty Ticker.news: list\n"

# Finnhub (works for US News)

In [31]:
import requests
r = requests.get("https://finnhub.io/api/v1/company-news",
                 params={"symbol":"AZN","from":"2025-10-10","to":"2025-11-09","token":FINNHUB_TOKEN})
articles = r.json()
print(f"Fetched {len(articles)} articles from Finnhub")

Fetched 161 articles from Finnhub


# NewsData.io
Gets regular news, but not financial news and needs company name instead of ticker

In [32]:
import requests
url = "https://newsdata.io/api/1/news"
#params = {"apikey": "YOUR_KEY", "q": "NESN.SW", "language": "en"}
params = {"apikey": NEWSDATA_API_KEY, "q": "Nestle"}
r = requests.get(url, params=params)
print(r.json())


{'status': 'success', 'totalResults': 306, 'results': [{'article_id': 'ff00a96436c9625851887ede967c26b0', 'link': 'https://ca.style.yahoo.com/utahs-best-suburb-perfect-wasatch-004500209.html', 'title': "Utah's Best Suburb Is A Perfect Wasatch Mountain City With Breathtaking Views And Canyon Access", 'description': 'Utah is becoming more and more popular as a place to visit and live, and this suburb is the perfect spot to explore with breathtaking views and canyon access.', 'content': 'ONLY AVAILABLE IN PAID PLANS', 'keywords': ['lifestyle'], 'creator': ['Islands'], 'language': 'english', 'country': ['canada'], 'category': ['sports'], 'pubDate': '2025-11-10 00:45:00', 'pubDateTZ': 'UTC', 'image_url': 'https://s.yimg.com/ny/api/res/1.2/VLZrodAwmbuHCkx7PEskOQ--/YXBwaWQ9aGlnaGxhbmRlcjt3PTEyNDI7aD02OTc7Y2Y9d2VicA--/https://media.zenfs.com/en/islands_international_279/39afaef7227d6f02057a5c222f87316f', 'video_url': None, 'source_id': 'yahoo', 'source_name': 'Yahoo! News', 'source_priority': 

# Marketaux

In [33]:
url = "https://api.marketaux.com/v1/news/all"
params = {"symbols": "NESN.SW", "filter_entities": "true", "api_token": MARKETAUX_TOKEN}
r = requests.get(url, params=params)
print(r.json())

{'meta': {'found': 812, 'returned': 3, 'limit': 3, 'page': 1}, 'data': [{'uuid': 'b2bd6b8e-f1e1-40cb-87c5-0dc539c94763', 'title': 'Nestlé sweetens digital future with upgrade to SAP S/4HANA Cloud Private Edition', 'description': 'SAP SE\xa0(NYSE: SAP) announced that Nestlé S.A., one of the world’s leading food and beverage companies, has completed its first major upgrade to SAP S/4HANA', 'keywords': '', 'snippet': 'SAP SE (NYSE: SAP) announced that Nestlé S.A., one of the world’s leading food and beverage companies, has completed its first major upgrade to SAP S/4HANA C...', 'url': 'https://retailtimes.co.uk/nestle-sweetens-digital-future-with-upgrade-to-sap-s-4hana-cloud-private-edition/', 'image_url': 'https://retailtimes.co.uk/wp-content/uploads/2023/01/android-chrome-192x192-1-75x75.png', 'language': 'en', 'published_at': '2025-10-31T09:37:46.000000Z', 'source': 'retailtimes.co.uk', 'relevance_score': None, 'entities': [{'symbol': 'NESN.SW', 'name': 'Nestlé S.A.', 'exchange': None,

In [37]:
import requests
# OR.PA, AZN.L, NESN.SW

url = "https://api.marketaux.com/v1/news/all"
params = {
    "symbols": "NESN.SW",
    "filter_entities": "true",
    "group_similar": "false",
    "language": "en",
    "published_after": "2025-09-30",
    "published_before": "2025-10-24",
    "limit": 100,     # harmless to ask higher, server will cap to your plan
    "page": 1,
    "api_token": MARKETAUX_TOKEN
}

all_articles = []

while True:
    r = requests.get(url, params=params, timeout=20)
    data = r.json()

    batch = data.get("data", [])
    meta = data.get("meta", {})
    all_articles.extend(batch)

    print(f"page {meta.get('page')} returned {meta.get('returned')} of {meta.get('found')} total")

    # use the API reported limit and returned, not your requested limit
    returned = int(meta.get("returned", 0))
    if returned == 0:
        break

    params["page"] = int(meta.get("page", params["page"])) + 1

print(f"Total collected: {len(all_articles)}")

page 1 returned 3 of 3 total
page 2 returned 0 of 3 total
Total collected: 3


- OR.PA = 1
- NESN.SW = 3
- AZN.L = 20

In [38]:
# print the first 2 articles
for article in all_articles[:2]:
    print(article)
    

{'uuid': '54200f63-6283-45df-aa61-aefc50168ab1', 'title': 'Nestlé S.A. (NEST:CA) Q3 2025 Sales Call Transcript', 'description': 'Nestlé S.A. (NEOE:NEST:CA) Q3 2025 Sales Call October 16, 2025 3:30 AM EDTCompany ParticipantsDavid Hancock - Head of Investor RelationsPhilipp Navratil -...', 'keywords': '', 'snippet': "Presentation\n\nDavid Hancock\n\nHead of Investor Relations\n\nGood morning, and welcome to Nestlé's 9-month sales call. I'm David Hancock, Head of Investor Relation...", 'url': 'https://seekingalpha.com/article/4830356-nestle-s-a-nest-ca-q3-2025-sales-call-transcript', 'image_url': 'https://static.seekingalpha.com/assets/og_image_1200-29b2bfe1a595477db6826bd2126c63ac2091efb7ec76347a8e7f81ba17e3de6c.png', 'language': 'en', 'published_at': '2025-10-16T12:26:37.000000Z', 'source': 'seekingalpha.com', 'relevance_score': None, 'entities': [{'symbol': 'NESN.SW', 'name': 'Nestlé S.A.', 'exchange': None, 'exchange_long': None, 'country': 'ch', 'type': 'equity', 'industry': 'Consume